# Train pipeline

This notebook takes the train and test data and does the following:

- Data processing (cleaning, feature engineering, scaling etc.) to prepare it for model training
- Trains a simple model
- Saves any artifacts that will be used during inference to disk

> This process will likely be captured in a Pipeline asset on Highwind

In [1]:
# Config
RANDOM_SEED = 42
ARTIFACT_SAVE_DIR = "../saved_model/"
TRAIN_DATA_PATH = "../data/train.csv"
TEST_DATA_PATH = "../data/test.csv"
TARGET_COLUMN = "target"
MODEL_ARGS = {
    "C": 1,
    "solver": "lbfgs",
    "fit_intercept": True,
    "random_state": 42,
}
PUSH_TO_HF = True # Whether to push to Hugging Face Hub or not
HF_REPO_NAME = "MelioAI/iris-classifier" # For pushing model to Hugging Face Hub 

In [2]:
import os
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd
import joblib
from skops import card, hub_utils
from tempfile import mkdtemp, mkstemp
from pathlib import Path

/Users/ruan/Documents/github/highwind-examples/iris-classification/.train.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load data

In [3]:
train_df = pd.read_csv(TRAIN_DATA_PATH)
train_df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,4.6,3.6,1.0,0.2,0
1,5.7,4.4,1.5,0.4,0
2,6.7,3.1,4.4,1.4,1


In [4]:
# Separate features and labels
X_train = train_df.copy()
y_train = X_train.pop(TARGET_COLUMN)

In [5]:
# Check shapes
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")

X_train: (120, 4)
y_train: (120,)


## Process data

### Optional steps

Insert any optional data processing steps here

In [6]:
# (Optional) Add data cleaning here

In [7]:
# (Optional) Add feature engineering here

### Feature scaling

In [8]:
# Initialise scaler and scale train features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [9]:
# Save the scaler for later use
save_scaler_path = os.path.join(ARTIFACT_SAVE_DIR, "scaler.joblib")
joblib.dump(scaler, save_scaler_path)

['../saved_model/scaler.joblib']

## Train model

In [10]:
# Define model
model = LogisticRegression(**MODEL_ARGS)

In [11]:
# Train model
model.fit(X_train_scaled, y_train)

LogisticRegression(C=1, random_state=42)

In [12]:
# Save the model for later use
save_model_path = os.path.join(ARTIFACT_SAVE_DIR, "model.joblib")
joblib.dump(model, save_model_path)

['../saved_model/model.joblib']

## (Optional) Save to Hugging Face Hub
Save trained model files to the Hugging Face Hub so that they can be downloaded later. In this step, we use the useful helper functions provided by the [`skops`](https://github.com/skops-dev/skops/tree/main) package.

If `PUSH_TO_HF` is enabled (see top of this notebook), this section will execute. Remember to log into Hugging Face with the CLI by running: `huggingface-cli login` otherwise this section won't work.

In [13]:
# Make temporary local repo dir
local_repo = Path("../hf-repo") # mkdtemp(prefix="skops-")

if PUSH_TO_HF:

    # Initialise HF repo
    hub_utils.init(
        model=Path(save_model_path),
        requirements=[
            f"scikit-learn=={sklearn.__version__}",
            f"joblib=={joblib.__version__}"
        ],
        dst=local_repo,
        task="tabular-classification",
        data=X_train.head(),
        model_format="pickle"
    )

    # Add feature scaler to repo
    hub_utils.add_files(save_scaler_path, dst=local_repo)

In [14]:
if PUSH_TO_HF:

    # Create and populate basic model card
    model_card = card.Card(model=model)
    metadata = card.metadata_from_config(local_repo / "config.json")
    
    # Add model card detail
    limitations = (
        "This model is made for the purposes of showing how to use Highwind only."
    )
    model_description = (
        "This is a Logistic Regression model trained on iris dataset. This model could be"
        " used to predict type of iris flower, given certain dimensions. This model is very basic and"
        " should only be used as an example of how to use Highwind."
    )
    model_card_authors = "MelioAI, ruanmelio"
    usage_code = """
```python
import joblib
from huggingface_hub import hf_hub_download

# Feature scaler
hf_hub_download("MelioAI/iris-classifier", "scaler.joblib")
scaler = joblib.load("scaler.joblib")

# Classifier model
hf_hub_download("MelioAI/iris-classifier", "model.joblib")
model = joblib.load("model.joblib")
```
"""
    model_card.add(
        folded=False,
        **{
            "Model Card Authors": model_card_authors,
            "Intended uses & limitations": limitations,
            "Model description": model_description,
            "Model description/Intended uses & limitations": limitations,
            "How to Get Started with the Model": usage_code
        },
    )

    # Add tags
    model_card.metadata.library_name = "sklearn"
    model_card.metadata.tags = ["sklearn", "tabular-classification"]

    # Save model card
    model_card.save(local_repo / "README.md")

In [15]:
# Remember to log into HF with the CLI by running: huggingface-cli login
if PUSH_TO_HF:

    # Push to HF Hub
    hub_utils.push(
        repo_id=HF_REPO_NAME,
        source=local_repo
    )

/Users/ruan/Documents/github/highwind-examples/iris-classification/.train.venv/lib/python3.9/site-packages/skops/hub_utils/_hf_hub.py:577: FutureWarning: Creating repos on hf.co is subject to strict rate limits now and therefore this feature is to be removed from this library in version 0.10. You can use tools directly available in the huggingface_hub library instead to create and push files.
  warnings.warn(


## (Optional) model evaluation

Using test set

In [16]:
# Remove scaler and model from memory to prove loading from disk works
del model
del scaler

In [17]:
# Load test data
test_df = pd.read_csv(TEST_DATA_PATH)
print(f"test_df: {test_df.shape}")

# Separate features and labels
X_test = test_df.copy()
y_test = X_test.pop(TARGET_COLUMN)

test_df: (30, 5)


In [18]:
# Run through same preprocessing steps
# Feature scaling
scaler = joblib.load(os.path.join(ARTIFACT_SAVE_DIR, "scaler.joblib"))
X_test_scaled = scaler.transform(X_test)

# Load model
model = joblib.load(os.path.join(ARTIFACT_SAVE_DIR, "model.joblib"))

In [19]:
# Make predictions on test set
y_pred = model.predict(X_test_scaled)

In [20]:
# Model evaluation report
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

